<a href="https://colab.research.google.com/github/NaKangHoon/MyFirstGitProject/blob/newbie/wineANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
# !pip install tqdm
import tensorflow as tf
print(tf.__version__)
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

2.17.0


In [78]:
class WineANN(tf.keras.Model):
    def __init__(self):
        super(WineANN, self).__init__()
        # 층을 몇개, 노드(결과값)는 몇개 만들지
        self.hidden1 = tf.keras.layers.Dense(units=8,
                                             activation='relu',
                                             kernel_initializer = self.weight_initialization(),
                                             bias_initializer = self.weight_initialization())
        self.hidden2 = tf.keras.layers.Dense(units=8,
                                             activation='relu',
                                             kernel_initializer = self.weight_initialization(),
                                             bias_initializer = self.weight_initialization())
        self.OUTPUT = tf.keras.layers.Dense(units=2,
                                            activation=None,
                                            kernel_initializer = self.weight_initialization(),
                                            bias_initializer = self.weight_initialization())






    def weight_initialization(self):
        return tf.keras.initializers.RandomNormal(mean = 0 , stddev = 1)


    def call(self, x, training = False):
        # 층에 순서대로 계산
        y1 = self.hidden1(x)
        y2 = self.hidden2(y1)
        logits = self.OUTPUT(y2)
        sy = tf.nn.softmax(logits)
        return logits, sy



In [79]:
@tf.function
def cross_entropy_loss(logits,y):
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = y))

learningRate = 0.001
optimizer = tf.optimizers.Adam(learningRate) # 미분법(경사하강법). Adagrad,Momentum
@tf.function
def backward(model, x, y):
    with tf.GradientTape() as grad:
        logits, sy = model(x, training = True) # 콜 함수 부르는곳
        loss = cross_entropy_loss(logits, y)
    gradients = grad.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

@tf.function
def accuracy(predY, y):
    correct_pred = tf.equal(tf.argmax(predY, axis=1), tf.argmax(y, axis=1))
    acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    return acc

In [80]:
def train_learning(epochs, frequency, trainDataliter,model,XTest,yTest):
    y_loss = []
    for epoch in tqdm(range(epochs)):
        bx, by = next(trainDataliter) #batch를 300개씩 계산
        loss = backward(model, bx, by) # 오차 300개의 평균을 한번에 계산
        y_loss.append(loss)
        if epoch % frequency == 0 :
            predY,sy = model(bx, training = False) # 콜함수 요청
            acc = accuracy(predY,by)
            print(f'epoch : {epoch+1} ==> loss : {loss:.4f}, accuracy : {acc:.4f}')
    predY, sy = model(XTest, training = False)
    acc = accuracy(predY, yTest)
    print(f'test accuracy : {acc:.4f}')
    return y_loss




In [81]:
red_path = "/content/drive/MyDrive/세종교육/winequality-red.csv"
white_path = "/content/drive/MyDrive/세종교육/winequality-white.csv"
red_df = pd.read_csv(red_path, sep=';')
red_df['label'] = 0
white_df = pd.read_csv(white_path, sep=';')
white_df['label'] = 1
wine_df = pd.concat([red_df, white_df])
wine_df.reset_index(drop=True, inplace=True)
matrix = wine_df.values
X = matrix[:, :-1]
X = (X-X.min(axis=0)) / (X.max(axis=0)-X.min(axis=0))
print(X)
y = matrix[:, -1]


XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size=0.3, random_state=42)
XTrain, XTest = XTrain.astype('float32'), XTest.astype('float32')

# One-hot encode y
yTrain = tf.one_hot(yTrain.flatten(), 2)
yTest = tf.one_hot(yTest.flatten(), 2)

train_data = tf.data.Dataset.from_tensor_slices((XTrain, yTrain))
train_data = train_data.shuffle(buffer_size=XTrain.shape[0]).batch(300).repeat()
trainDataliter = iter(train_data)

epochs = 5000
frequency = 200
model = WineANN()
y_loss = train_learning(epochs, frequency, trainDataliter, model, XTest, yTest)


[[0.29752066 0.41333333 0.         ... 0.19101124 0.20289855 0.33333333]
 [0.33057851 0.53333333 0.         ... 0.25842697 0.26086957 0.33333333]
 [0.33057851 0.45333333 0.02409639 ... 0.24157303 0.26086957 0.33333333]
 ...
 [0.2231405  0.10666667 0.11445783 ... 0.13483146 0.20289855 0.5       ]
 [0.14049587 0.14       0.18072289 ... 0.08988764 0.69565217 0.66666667]
 [0.18181818 0.08666667 0.22891566 ... 0.05617978 0.55072464 0.5       ]]


  0%|          | 1/5000 [00:02<3:38:59,  2.63s/it]

epoch : 1 ==> loss : 2.0720, accuracy : 0.2367


  4%|▍         | 211/5000 [00:05<00:51, 93.78it/s]

epoch : 201 ==> loss : 0.4845, accuracy : 0.7933


...

epoch : 4601 ==> loss : 0.0363, accuracy : 0.9933


 98%|█████████▊| 4880/5000 [00:19<00:00, 439.17it/s]

epoch : 4801 ==> loss : 0.0260, accuracy : 0.9900


100%|██████████| 5000/5000 [00:19<00:00, 252.01it/s]


test accuracy : 0.9867
